# Neural Network 

Fingerprint: Coulomb

In [91]:
from Coulomb import *
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold  
import joblib  # For saving and loading scaler


In [92]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=251)


In [93]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Normalize the target (hform)
#target_scaler = MinMaxScaler()  # You can use StandardScaler if needed
#y_train = target_scaler.fit_transform(y_train.reshape(-1, 1) if isinstance(y_train, np.ndarray) else y_train.to_numpy().reshape(-1, 1))
#y_test = target_scaler.transform(y_test.reshape(-1, 1) if isinstance(y_test, np.ndarray) else y_test.to_numpy().reshape(-1, 1))

# Ensure y_train and y_test are properly converted to NumPy arrays
X_train = torch.tensor(X_train, dtype=torch.float32)

# Convert y_train and y_test to NumPy arrays if they are Series or other objects
if isinstance(y_train, pd.Series):
    y_train = y_train.to_numpy()

if isinstance(y_test, pd.Series):
    y_test = y_test.to_numpy()

# Convert to PyTorch tensors
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Add dimension
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Add dimension


In [94]:
# Define the neural network
class RegressionNN(nn.Module):
    def __init__(self, input_dim):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Increased neurons
        self.bn1 = nn.BatchNorm1d(256)  # Batch normalization
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(p=0.2)  # Dropout to reduce overfitting

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))  # LeakyReLU activation
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x



In [101]:
def cross_val_train(model_class, X_train, y_train, epochs, k_folds, patience=50):
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_results = []
    best_overall_val_loss = float('inf')
    best_overall_model_state = None

    if not isinstance(X_train, torch.Tensor):
        X_train = torch.tensor(X_train, dtype=torch.float32)
    if not isinstance(y_train, torch.Tensor):
        y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        print(f"\nFold {fold + 1}/{k_folds}")

        X_fold_train = X_train[train_idx]
        y_fold_train = y_train[train_idx]
        X_val = X_train[val_idx]
        y_val = y_train[val_idx]

        model = model_class(X_train.shape[1])
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=True)
        criterion = nn.MSELoss()

        best_fold_val_loss = float('inf')
        best_fold_model_state = None
        patience_counter = 0
        best_epoch_rmse = float('inf')

        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_fold_train)
            loss = criterion(outputs, y_fold_train)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            model.eval()
            with torch.no_grad():
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, y_val)

            scheduler.step(val_loss)

            if val_loss < best_fold_val_loss:
                best_fold_val_loss = val_loss
                best_fold_model_state = model.state_dict()
                patience_counter = 0
                best_epoch_rmse = torch.sqrt(val_loss).item()
            else:
                patience_counter += 1

            train_rmse = torch.sqrt(loss).item()
            val_rmse = torch.sqrt(val_loss).item()

            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{epochs}], Train RMSE: {train_rmse:.4f}, Val RMSE: {val_rmse:.4f}")

        fold_results.append(best_epoch_rmse)

        if best_fold_val_loss < best_overall_val_loss:
            best_overall_val_loss = best_fold_val_loss
            best_overall_model_state = best_fold_model_state.copy()
            print(f"New best model found in fold {fold + 1}")

    print("\nCross-Validation Results:")
    print(f"Fold RMSEs: {[f'{rmse:.4f}' for rmse in fold_results]}")
    print(f"Mean RMSE: {np.mean(fold_results):.4f}")
    print(f"Standard Deviation: {np.std(fold_results):.4f}")

    # Save only the model state
    torch.save(best_overall_model_state, "best_model_state.pth")
    print("Best model state saved as 'best_model_state.pth'.")

    return best_overall_model_state, fold_results


In [102]:
# Initialize loss function
criterion = nn.MSELoss()

# Perform cross-validation
cross_val_train(RegressionNN, X_train, y_train, epochs=50, k_folds=3, patience=50)
# Save the model
# Example usage: torch.save(model.state_dict(), "enhanced_regression_model.pth")





Fold 1/3
New best model found in fold 1

Fold 2/3

Fold 3/3

Cross-Validation Results:
Fold RMSEs: ['0.5863', '0.5991', '0.5887']
Mean RMSE: 0.5914
Standard Deviation: 0.0056
Best model state saved as 'best_model_state.pth'.


(OrderedDict([('fc1.weight',
               tensor([[-1.3519e-03, -6.1236e-03, -1.7919e-03,  ...,  9.0031e-03,
                        -4.8412e-03, -9.8319e-03],
                       [-1.6846e-02, -1.0922e-02,  2.7495e-02,  ..., -7.7153e-03,
                        -1.7040e-03,  1.1466e-03],
                       [-4.9724e-03,  7.7742e-03,  3.3037e-03,  ...,  7.3299e-04,
                        -3.2818e-03, -2.8968e-03],
                       ...,
                       [-7.7775e-03,  2.0717e-02, -1.7368e-03,  ..., -9.8619e-03,
                        -1.6995e-02, -2.6100e-03],
                       [-1.5317e-03,  2.6319e-02,  2.7501e-03,  ...,  2.5388e-02,
                         9.2079e-03, -1.0185e-02],
                       [ 1.6772e-02, -1.2983e-02, -2.6280e-02,  ..., -7.0048e-03,
                        -4.2891e-05,  2.4481e-03]])),
              ('fc1.bias',
               tensor([-8.2353e-03,  4.9515e-03, -6.6919e-04, -1.4332e-03,  1.9930e-03,
                        7.3

In [103]:
# Load the best model and train it on the full training set
def train_on_full_data(model_class, X_train, y_train, X_test, y_test, criterion, epochs=100,  patience=100):
    model = model_class(X_train.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=100, verbose=True)
    # Load the best model state
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0

    model.load_state_dict(torch.load("best_model.pth"))

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_test)
            val_loss = criterion(val_outputs, y_test)



        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch}")
            break
        if (epoch + 1) % 100 == 0:
            rmse = torch.sqrt(loss).item()
            print(f"Epoch [{epoch + 1}/{epochs}], RMSE: {rmse:.4f}")

    print("Training on full dataset completed.")

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)
        test_rmse = torch.sqrt(test_loss).item()
        print(f"\nTest RMSE: {test_rmse:.4f}")

    # Save the final model
    torch.save(model.state_dict(), "final_model.pth")
    print("Final model saved as 'final_model.pth'.")

# Train the best model on the full training set and evaluate on test set
train_on_full_data(RegressionNN, X_train, y_train, X_test, y_test, criterion, epochs=1000)

C:\Users\Antonia\AppData\Local\Temp\ipykernel_9536\2036549851.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


RuntimeError: Error(s) in loading state_dict for RegressionNN:
	Missing key(s) in state_dict: "fc1.weight", "fc1.bias", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "fc2.weight", "fc2.bias", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "fc3.weight", "fc3.bias", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var", "fc4.weight", "fc4.bias", "fc5.weight", "fc5.bias". 
	Unexpected key(s) in state_dict: "model_state", "fold_results", "mean_rmse", "std_rmse". 

[0.11333410441875458]
[0.11041968315839767]